In [11]:
import json
import openai

Gives us all the claims and all of their decompositions to use in prompting

In [12]:

with open("../../data/ClaimDecomp/train_claimdecomp_evidence_question_mapping.json", "r", encoding="utf-8") as f:
    train = json.load(f)

In [13]:
all_claims = [(i, claim['claim']) for i, claim in enumerate(train)]

In [33]:
train[9834]

{'claim': 'Says “Hunter Biden had 25,000 pics of him torturing and raping children under age 10 in China on his laptop.”',
 'evidences': [{'questions': 'did mike lindell blame "antifa temptresses" for the jan. 6, 2021 attack on the u.s. capitol?\n',
   'top_k_doc': ['this tendency was amplified in the wake of the january 6th capitol insurrection, as almost immediately false claims surfaced that antifa was the first to storm the capitol  not pro-trump extremists. these allegations were spread even by mainstream right-wing media personalities who falsely claimed that there were reports of busloads of antifa infiltrating the ranks of the pro-trump crowd and that it was antifa that was the first to break the police barrier. on december 6, 2021, prosecutors charged seven individuals described as self-identified anti-fascists related to assaults that took place in january during the patriot march at pacific beach in january 2021. the criminal complaint alleges that the self-identified antifa

In [34]:
train[9839]

{'claim': 'Former president Barack Obama referred to himself 75 times during his farewell speech, compared to just 3 self-references by President Donald Trump in his inaugural address.',
 'evidences': [{'questions': 'did barack obama refer to himself 75 times during his farewell speech?\n',
   'top_k_doc': ['barack obama gave a farewell speech, stating many achievements made during ... bush, barack obama, and donald trump, 19932017." journal of contextual ...']},
  {'questions': ' did president donald trump make only 3 self-references in his inaugural address?\n',
   'top_k_doc': ["jan 18, 2018  president donald trump gives his inaugural address(photo by samuel corum/anadolu agency/getty images). one year ago, president donald trump's ..."]},
  {'questions': ' is there a significant difference in the number of self-references made by barack obama and donald trump in their respective speeches?',
   'top_k_doc': ['applying this guideline to trumps speech, its pretty clear that she plagia

In [15]:
all_claims_decomps = []
for claim_id, i in enumerate(train):
    output_dict = {}
    output_dict['claim_id'] = claim_id
    output_dict['original_claim'] = i['claim']
    
    for decomp_id, j in enumerate(i['evidences']):
        output_dict[f'question{decomp_id}']= j['questions']

    all_claims_decomps.append(output_dict)

In [17]:
all_data = [{'claim_id':i['claim_id'], 'original_claim':i['original_claim']} for i in all_claims_decomps]

In [28]:
from openai import OpenAI

client = OpenAI(api_key="passkey")
import json
import time
from tqdm import tqdm

# -----------------------------
# USER CONFIGURATION
# -----------------------------

INPUT_FILE = "claims.json"          # Your input file with claims data
OUTPUT_FILE = "claims_decomposed.json"  # Output file for the decomposed claims

# -----------------------------
# FEW-SHOT EXAMPLES  
# -----------------------------
few_shot_examples = [
    {
        "role": "assistant",
        "content": (
            "{\n"
            '  "claim_id": 0,\n'
            '  "original_claim": "In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.",\n'
            '  "question0": "Did Nirmala Sitharaman claim that the government distributed 35,000 crore LED bulbs?",\n'
            '  "question1": "Is there evidence that the government distributed 35,000 crore LED bulbs?",\n'
            '  "question2": "Are there records to support the distribution of 35,000 crore LED bulbs?"\n'
            "}"
        )
    },
    {
        "role": "assistant",
        "content": (
            "{\n"
            '  "claim_id": 1,\n'
            '  "original_claim": "Florida residents affected by Hurricane Irma can receive $197 in food stamp benefits, but only if they can show that their homes lost power for more than two hours.",\n'
            '  "question0": "Do Florida residents affected by Hurricane Irma qualify for $197 in food stamp benefits?",\n'
            '  "question1": "Is it necessary for them to prove that their homes lost power for more than two hours?",\n'
            '  "question2": "Is the benefit exclusively available to those affected by Hurricane Irma?"\n'
            "}"
        )
    }
]

# -----------------------------
# SYSTEM MESSAGE (General Instructions)
# -----------------------------
system_message = {
    "role": "system",
    "content": (
        "You are an assistant that, given a claim, generates up to 3 yes/no questions to aid in fact verification. "
        "Return your answer in valid JSON with the following keys: 'claim_id', 'original_claim', 'question0', 'question1', and 'question2'. "
        "Do not include any additional commentary."
    )
}

# -----------------------------
# LOAD INPUT DATA
# -----------------------------
# with open(INPUT_FILE, "r", encoding="utf-8") as f:
#     input_data = json.load(f)

output_data = []

# -----------------------------
# PROCESS EACH CLAIM
# -----------------------------
for item in tqdm(all_data):
    claim_id = item.get("claim_id")
    original_claim = item.get("original_claim")

    # Build conversation history with the system message, few-shot examples, and the test claim.
    messages = [system_message] + few_shot_examples + [
        {
            "role": "user",
            "content": f"Claim ID: {claim_id}\nOriginal Claim: {original_claim}"
        }
    ]

    try:
        response = client.chat.completions.create(model="gpt-4o-mini",  # Updated model
        messages=messages,
        temperature=0.3,             # Lower temperature for more deterministic output
        max_tokens=300,
        frequency_penalty=0.6,       # Penalizes repetition of the same phrases
        presence_penalty=0.8,        # Encourages new ideas by penalizing already-mentioned tokens
        n=1)
        assistant_message = response.choices[0].message.content.strip()

        try:
            result_json = json.loads(assistant_message)
        except json.JSONDecodeError:
            # Fallback in case the response isn't valid JSON
            result_json = {
                "claim_id": claim_id,
                "original_claim": original_claim,
                "question0": assistant_message,
                "question1": "",
                "question2": ""
            }

        output_data.append(result_json)
        # Pause briefly to avoid hitting rate limits
        time.sleep(1)

    except Exception as e:
        print(f"Error processing claim_id {claim_id}: {e}")
        output_data.append({ 
            "claim_id": claim_id,
            "original_claim": original_claim,  
            "question0": "",
            "question1": "",
            "question2": ""
        })

# -----------------------------
# SAVE OUTPUT DATA
# -----------------------------
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"Processing complete. Check '{OUTPUT_FILE}' for results.")


100%|██████████| 9935/9935 [10:31:54<00:00,  3.82s/it]    

Processing complete. Check 'claims_decomposed.json' for results.
